# Mapping New York

The goal of this notebook is to map NY State using Kepler, visually representin population density using different colors. We'll add a focus line around NYC. Using the GEO DataFrame of NY State census and NY State places, selecting name “New York City”, from the previous notebook.

In [1]:
import os, pathlib
base_dir = pathlib.Path(os.getcwd()).parent
data_archive_dir = os.path.join(base_dir, "data_archive")
clean_data_dir = os.path.join(data_archive_dir, "clean")
data_dir = os.path.join(base_dir, "data")
shapes_dir = os.path.join(data_dir,"shapes")
json_dir = os.path.join(data_dir,"geojson")

In [2]:
import pandas as pd
import geopandas as gpd

In [3]:
# import our GEO DataFrame of NY State Census Tracts 
clean_geodf_tract_ny_file = os.path.join(clean_data_dir,'geodf_tract_ny.csv')
geodf_tract_ny = pd.read_csv(clean_geodf_tract_ny_file, index_col=0)
geodf_tract_ny.head(3)

,State FIPS,County FIPS,Census Tract Name,GEOID Census Tract,Population,Land Area,Population Density,geometry
0,36,47,"Census Tract 7, Kings County, New York",36047000700,4415,176774,64686,"POLYGON ((-74.001542 40.692786, -74.001317 40...."
1,36,47,"Census Tract 9, Kings County, New York",36047000900,5167,163469,81865,"POLYGON ((-73.994046 40.690897, -73.993736 40...."
2,36,47,"Census Tract 11, Kings County, New York",36047001100,1578,168507,24254,"POLYGON ((-73.99072799999999 40.693048, -73.99..."


In [4]:
from keplergl import KeplerGl
ny_map = KeplerGl(height=600, show_docs=False)
ny_map.add_data(data=geodf_tract_ny)
ny_map

KeplerGl(data={'unnamed': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,…

In [5]:
# import our GEO DataFrame of NY State Places
clean_geodf_place_ny_file = os.path.join(clean_data_dir,'geodf_place_ny.csv')
geodf_place_ny = pd.read_csv(clean_geodf_place_ny_file, index_col=0)
# geodf_place_ny.head(3)

In [14]:
# ny_map = KeplerGl(height=600, show_docs=False)
ny_map = KeplerGl(height=600, show_docs=False)
ny_map.add_data(data=geodf_place_ny)
ny_map

KeplerGl(data={'unnamed': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,…

### Outline the place "New York City"

For some reason, the place names are inconsistent between our NY places shapefiles and our NY places list. Specifically, Place FIPS 51000 is referred to by our places list as "New York City", while as we can see below, our shapefile refers to it as "New York". This is confusing! We like the version in our places list and would rather use this. So, let's merge *shapefile_place_ny* with *df_place* to bring in the "Place" column we like from df_place.

In [7]:
geodf_place_nyc = geodf_place_ny[geodf_place_ny['Place FIPS'] == 51000]

In [8]:
import pyarrow.parquet as pq

clean_df_place_file = os.path.join(clean_data_dir, 'place.parquet')
df_place = pq.read_table(clean_df_place_file, memory_map=True).to_pandas()
df_place.head(3)

,State Name,State,State FIPS,Place,Place FIPS,County
0,Alabama,AL,1,Abanda Cdp,100,Chambers County
1,Alabama,AL,1,Abbeville City,124,Henry County
2,Alabama,AL,1,Adamsville City,460,Jefferson County


In [9]:
# don't include "name"
geodf_place_ny = geodf_place_ny.merge(df_place, on =['State FIPS', 'Place FIPS'])
geodf_place_ny = geodf_place_ny[['State Name', 'State', 'State FIPS', 'Place', 'Place FIPS', 'County', 'Land Area', 'geometry']]
geodf_place_ny.head(3)

,State Name,State,State FIPS,Place,Place FIPS,County,Land Area,geometry
0,New York,NY,36,Altamont Village,1517,Albany County,3279144,"MULTIPOLYGON (((-74.022713 42.704198999999996,..."
1,New York,NY,36,Peekskill City,56979,Westchester County,11251161,"POLYGON ((-73.955445 41.277862, -73.952045 41...."
2,New York,NY,36,Peekskill City,56979,Westchester County,11251161,"POLYGON ((-73.955445 41.277862, -73.952045 41...."


In [10]:
# make a new GEO DF of just the Place New York City?
geodf_place_nyc = geodf_place_ny[geodf_place_ny['Place'] == 'New York City']
geodf_place_nyc

,State Name,State,State FIPS,Place,Place FIPS,County,Land Area,geometry
440,New York,NY,36,New York City,51000,"Bronx County, Kings County, New York County, Q...",778165283,"MULTIPOLYGON (((-74.040747 40.700167, -74.0407..."


In [15]:
### I want to add an outline of NYC

ny_map = KeplerGl(height=600, show_docs=False)
ny_map.add_data(geodf_place_nyc, name='New York City') 

# geodf_place_nyc["geometry"].explode().iloc[3]
# geodf_place_nyc["geometry"].boundary.plot()

ny_map

KeplerGl(data={'New York City': {'index': [440], 'columns': ['State Name', 'State', 'State FIPS', 'Place', 'Pl…

In [12]:
ny_map.config = {
    "version": "v1",
    "config": {
        "visState": {
            "filters": [],
            "layers": [
                {
                    "id": "5xtj533",
                    "type": "geojson",
                    "config": {
                        "dataId": "New York City",
                        "label": "New York City",
                        "color": [
                            183,
                            136,
                            94
                        ],
                        "highlightColor": [
                            252,
                            242,
                            26,
                            255
                        ],
                        "columns": {
                            "geojson": "geometry"
                        },
                        "isVisible": True,
                        "visConfig": {
                            "opacity": 0.8,
                            "strokeOpacity": 0.8,
                            "thickness": 1.6,
                            "strokeColor": [
                                125,
                                201,
                                127
                            ],
                            "colorRange": {
                                "name": "Global Warming",
                                "type": "sequential",
                                "category": "Uber",
                                "colors": [
                                    "#5A1846",
                                    "#900C3F",
                                    "#C70039",
                                    "#E3611C",
                                    "#F1920E",
                                    "#FFC300"
                                ]
                            },
                            "strokeColorRange": {
                                "name": "Global Warming",
                                "type": "sequential",
                                "category": "Uber",
                                "colors": [
                                    "#5A1846",
                                    "#900C3F",
                                    "#C70039",
                                    "#E3611C",
                                    "#F1920E",
                                    "#FFC300"
                                ]
                            },
                            "radius": 10,
                            "sizeRange": [
                                0,
                                10
                            ],
                            "radiusRange": [
                                0,
                                50
                            ],
                            "heightRange": [
                                0,
                                500
                            ],
                            "elevationScale": 5,
                            "enableElevationZoomFactor": True,
                            "stroked": True,
                            "filled": False,
                            "enable3d": False,
                            "wireframe": False
                        },
                        "hidden": False,
                        "textLabel": [
                            {
                                "field": None,
                                "color": [
                                    255,
                                    255,
                                    255
                                ],
                                "size": 18,
                                "offset": [
                                    0,
                                    0
                                ],
                                "anchor": "start",
                                "alignment": "center"
                            }
                        ]
                    },
                    "visualChannels": {
                        "colorField": None,
                        "colorScale": "quantile",
                        "strokeColorField": None,
                        "strokeColorScale": "quantile",
                        "sizeField": None,
                        "sizeScale": "linear",
                        "heightField": None,
                        "heightScale": "linear",
                        "radiusField": None,
                        "radiusScale": "linear"
                    }
                },
                {
                    "id": "3xpf35a",
                    "type": "geojson",
                    "config": {
                        "dataId": "NY State",
                        "label": "NY State",
                        "color": [
                            23,
                            184,
                            190
                        ],
                        "highlightColor": [
                            252,
                            242,
                            26,
                            255
                        ],
                        "columns": {
                            "geojson": "geometry"
                        },
                        "isVisible": True,
                        "visConfig": {
                            "opacity": 0.8,
                            "strokeOpacity": 0.8,
                            "thickness": 0.5,
                            "strokeColor": [
                                246,
                                209,
                                138
                            ],
                            "colorRange": {
                                "name": "Global Warming",
                                "type": "sequential",
                                "category": "Uber",
                                "colors": [
                                    "#5A1846",
                                    "#900C3F",
                                    "#C70039",
                                    "#E3611C",
                                    "#F1920E",
                                    "#FFC300"
                                ]
                            },
                            "strokeColorRange": {
                                "name": "Global Warming",
                                "type": "sequential",
                                "category": "Uber",
                                "colors": [
                                    "#5A1846",
                                    "#900C3F",
                                    "#C70039",
                                    "#E3611C",
                                    "#F1920E",
                                    "#FFC300"
                                ]
                            },
                            "radius": 10,
                            "sizeRange": [
                                0,
                                10
                            ],
                            "radiusRange": [
                                0,
                                50
                            ],
                            "heightRange": [
                                0,
                                500
                            ],
                            "elevationScale": 5,
                            "enableElevationZoomFactor": True,
                            "stroked": False,
                            "filled": True,
                            "enable3d": False,
                            "wireframe": False
                        },
                        "hidden": False,
                        "textLabel": [
                            {
                                "field": None,
                                "color": [
                                    255,
                                    255,
                                    255
                                ],
                                "size": 18,
                                "offset": [
                                    0,
                                    0
                                ],
                                "anchor": "start",
                                "alignment": "center"
                            }
                        ]
                    },
                    "visualChannels": {
                        "colorField": {
                            "name": "Population Density",
                            "type": "integer"
                        },
                        "colorScale": "quantile",
                        "strokeColorField": None,
                        "strokeColorScale": "quantile",
                        "sizeField": None,
                        "sizeScale": "linear",
                        "heightField": None,
                        "heightScale": "linear",
                        "radiusField": None,
                        "radiusScale": "linear"
                    }
                }
            ],
            "interactionConfig": {
                "tooltip": {
                    "fieldsToShow": {
                        "NY State": [
                            {
                                "name": "State FIPS",
                                "format": None
                            },
                            {
                                "name": "County FIPS",
                                "format": None
                            },
                            {
                                "name": "Census Tract Code",
                                "format": None
                            },
                            {
                                "name": "GEOID Census Tract",
                                "format": None
                            },
                            {
                                "name": "Land Area",
                                "format": None
                            }
                        ],
                        "New York City": [
                            {
                                "name": "State Name",
                                "format": None
                            },
                            {
                                "name": "State",
                                "format": None
                            },
                            {
                                "name": "State FIPS",
                                "format": None
                            },
                            {
                                "name": "Place",
                                "format": None
                            },
                            {
                                "name": "Place FIPS",
                                "format": None
                            }
                        ]
                    },
                    "compareMode": False,
                    "compareType": "absolute",
                    "enabled": True
                },
                "brush": {
                    "size": 0.5,
                    "enabled": False
                },
                "geocoder": {
                    "enabled": False
                },
                "coordinate": {
                    "enabled": False
                }
            },
            "layerBlending": "additive",
            "splitMaps": [],
            "animationConfig": {
                "currentTime": None,
                "speed": 1
            }
        },
        "mapState": {
            "bearing": 0,
            "dragRotate": False,
            "latitude": 42.93740667647843,
            "longitude": -77.20412718754224,
            "pitch": 0,
            "zoom": 6,
            "isSplit": False
        },
        "mapStyle": {
            "styleType": "dark",
            "topLayerGroups": {},
            "visibleLayerGroups": {
                "label": True,
                "road": True,
                "border": False,
                "building": True,
                "water": True,
                "land": True,
                "3d building": False
            },
            "threeDBuildingColor": [
                9.665468314072013,
                17.18305478057247,
                31.1442867897876
            ],
            "mapStyles": {}
        }
    }
}


In [13]:
# Uncomment this to show the current configuration of the map 
# after you play with the settings (but before you reload)
#ny_map.config